In [1]:
from appgeopy import *
from my_packages import *

In [2]:
config_fpath = r"D:\VENV_PYTHON\insar_scriptflow\C__calibration_config.ini"
config = configparser.ConfigParser()
config.read(config_fpath)

['D:\\VENV_PYTHON\\insar_scriptflow\\C__calibration_config.ini']

In [3]:
# Extract general parameters from the configuration file
workflow_step = config["GENERAL"]["workflow_step"]
mainfolder = config["GENERAL"]["insstackpsi_mainfolder"]

# Define paths for post-processing folders

los_difference_output = config["GPS_DIFFERENCE"]["los_difference_output"]

# Assume the script A1 have been run already
calibration_fld = os.path.join(mainfolder, "15_CALIBRATION")
config_fld = os.path.join(calibration_fld, "configs")

# Ensure the config folder exists and save the used configuration file
os.makedirs(config_fld, exist_ok=True)
# data_io.save_configs(src_file=args.configfile, dest_folder=config_fld)

prepared_los_difference_outname = "PREPARED_" + los_difference_output
prepared_los_difference_fpath = os.path.join(
    calibration_fld, prepared_los_difference_outname
)

horizontal_movement_folder = config["PRE_KRIGING"]["horizontal_movement_folder"]
horizontal_movement_files = glob(
    horizontal_movement_folder + r"\PREPARED*.xlsx"
)

file2process = [prepared_los_difference_fpath] + horizontal_movement_files
file2process

['E:\\030_CHOUSHUI_2024\\000_INSCALDEFO_2_INSSTACKPSI\\PROCESS_004\\15_CALIBRATION\\PREPARED_InSAR_GPS_LOS_Difference.xlsx',
 'E:\\SUBSIDENCE_PROJECT_DATA\\GPS_2023_YehDaGang\\2023_NEW_20240515\\20240527_PROCESSED_DATA\\_MERGED\\PREPARED_EW_Difference_mm_2023_2023.xlsx',
 'E:\\SUBSIDENCE_PROJECT_DATA\\GPS_2023_YehDaGang\\2023_NEW_20240515\\20240527_PROCESSED_DATA\\_MERGED\\PREPARED_NS_Difference_mm_2023_2023.xlsx']

In [7]:
# select_file = file2process[0]

for select_file in file2process:

    prepared_df = pd.read_excel(select_file)

    value_columns = prepared_df.columns[3:]

    # Create a new DataFrame to hold the transformed data values
    output_records = prepared_df.iloc[:, [0, 1, 2]].copy()
    output_records = output_records.set_index("STATION")

    for target_column in value_columns:
        # Select the data to be transformed
        temp = prepared_df.loc[:, ["STATION", target_column]].copy()
        temp = temp[prepared_df.loc[:, target_column].notnull()]

        transformed_column = target_column.replace("N", "Trans")

        # Apply normal score transform to the selected data
        temp[transformed_column], tvDisp, tnsDisp = geostats.nscore(
            temp, target_column
        )

        # Apply filter to remove outliers
        # filter_cond = (temp[transformed_column] >= -3) & (
        #     temp[transformed_column] <= 3
        # )

        filter_cond = (temp[transformed_column] >= -2) & (
            temp[transformed_column] <= 2
        )

        temp = temp[filter_cond]

        temp = temp.set_index("STATION")
        
        output_records[transformed_column] = output_records.index.map(
            temp[transformed_column]
        )

    output_records_savepath = os.path.join(
        calibration_fld,
        os.path.basename(select_file).replace("PREPARED", "TRANSFORMED"),
    )
    output_records_savepath = output_records_savepath.replace("xlsx", "csv")

    output_records = output_records.reset_index(drop=False)
    output_records.to_csv(output_records_savepath, drop=False)

    print(
        f"\n{os.path.basename(select_file)} has been saved at\n{output_records_savepath}"
    )


PREPARED_InSAR_GPS_LOS_Difference.xlsx has been saved at
E:\030_CHOUSHUI_2024\000_INSCALDEFO_2_INSSTACKPSI\PROCESS_004\15_CALIBRATION\TRANSFORMED_InSAR_GPS_LOS_Difference.csv

PREPARED_EW_Difference_mm_2023_2023.xlsx has been saved at
E:\030_CHOUSHUI_2024\000_INSCALDEFO_2_INSSTACKPSI\PROCESS_004\15_CALIBRATION\TRANSFORMED_EW_Difference_mm_2023_2023.csv

PREPARED_NS_Difference_mm_2023_2023.xlsx has been saved at
E:\030_CHOUSHUI_2024\000_INSCALDEFO_2_INSSTACKPSI\PROCESS_004\15_CALIBRATION\TRANSFORMED_NS_Difference_mm_2023_2023.csv
